# Part-II: VacationPy

In [1]:
# Dependencies and Setup
from IPython.display import display
import matplotlib.pyplot as plt
import scipy.stats as st
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from ipywidgets.embed import embed_minimal_html

# Import 'API' and 'G' Keys
from config import gkey
from config import (api_key)

# Configure 'G' Key
gmaps.configure(api_key = gkey)

### Store Part-I results into DataFrame
* Load the CSV that was exported in Part-I, to a DataFrame

In [2]:
# Dependencies and Setup
pro_dir = ''

cities_path = "../selected_cities/selected_cities.csv"

# Read the 'Cities' data that we just stored in a CSV file
cities_data = pd.read_csv(pro_dir + cities_path)

locations = cities_data[['Latitude', 'Longitude']]
hum_weights = cities_data['Humidity (%)']
fig = gmaps.figure(display_toolbar = True, center = (0.0, 0.0), display_errors = True, zoom_level = 1)
fig.add_layer(gmaps.heatmap_layer(locations, weights = hum_weights))
embed_minimal_html("maps/map1.html", views = [fig])
fig

Figure(layout=FigureLayout(height='420px'))

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
cities_utemp_limit = cities_data[cities_data['Max. Temp (F)'] < 80]
cities_ultemp_limit = cities_utemp_limit[cities_utemp_limit['Max. Temp (F)'] > 70]
cities_ftemp_fws = cities_ultemp_limit[cities_ultemp_limit['Wind Speed (mph)'] < 10]
cities_filtered = cities_ftemp_fws[cities_ftemp_fws['Cloudiness (%)'] == 0]
display(cities_filtered)
targ_rad = 5000

,City Name,Latitude,Longitude,Temperature (F),Max. Temp (F),Min. Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country Code,Date & Time
41,caravelas,-17.7125,-39.2481,72.21,72.21,72.21,90,0,2.01,BR,14-01-2021 17:37:17
130,cape town,-33.9258,18.4232,73.58,75.99,71.01,73,0,4.61,ZA,14-01-2021 17:25:42
239,nizwa,22.9333,57.5333,74.84,74.84,74.84,19,0,5.26,OM,14-01-2021 17:41:43
300,wellington,-41.2866,174.7756,67.33,71.01,66.00,77,0,8.05,NZ,14-01-2021 17:43:08
384,ouadda,8.0777,22.4007,79.30,79.30,79.30,20,0,5.95,CF,14-01-2021 17:45:08
461,sur,22.5667,59.5289,74.73,74.73,74.73,44,0,8.03,OM,14-01-2021 17:46:52


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [4]:
final_rest_data = {'Latitude':[],
                   'Longitude':[],
                   'Restaurant (Eatery) Name':[],
                   'City':[],
                   'Country Code':[]
                  }

for index, row in cities_filtered.iterrows():
    coordinates = str(row['Latitude']) + ', ' + str(row['Longitude'])
    params = {"location": coordinates,
              "type": "restaurant",
              'radius': targ_rad,
              "key": gkey
             }
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(url, params = params)
    restaurants_places = response.json()
    
    try:
        rest_data = restaurants_places['results'][0]
        rs_nm = rest_data['name']
        final_rest_data['Restaurant (Eatery) Name'].append(rest_data['name'])
        final_rest_data['Latitude'].append(restaurants_places['results'][0]['geometry']['location']['lat'])
        final_rest_data['Longitude'].append(restaurants_places['results'][0]['geometry']['location']['lng'])
        final_rest_data['City'].append(row['City Name'])
        final_rest_data['Country Code'].append(row['Country Code'])
    except:
        aaa = 0
hotel_df = pd.DataFrame(final_rest_data)
display(hotel_df)

,Latitude,Longitude,Restaurant (Eatery) Name,City,Country Code
0,-17.736028,-39.262938,Restaurante Carenagem,caravelas,BR
1,-33.915763,18.423496,Southern Sun Waterfront Cape Town,cape town,ZA
2,22.932472,57.533966,Al-Mandi Al-Dhahabi Restaurant,nizwa,OM
3,-41.285861,174.774831,Travelodge Hotel Wellington,wellington,NZ
4,22.575852,59.502457,Pizza Hut,sur,OM


In [5]:
# NOTE: Do not change any of the code in this cell
# Using the template, add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Restaurant (Eatery) Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country Code}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: Be sure to update with your DataFrame name

hotel_info_b = ['Name \n' + row['Restaurant (Eatery) Name'] + '\n City \n' + row['City'] + '\n Country \n' + row['Country Code'] for index, row in hotel_df.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]
marker_layer = gmaps.marker_layer(locations, hover_text = hotel_info_b)
fig = gmaps.figure(display_toolbar = True, center = (0.0, 0.0), display_errors = True, zoom_level = 1)
#fig = gmaps.figure()
fig.add_layer(marker_layer)
embed_minimal_html("maps/map2.html", views = [fig])
fig

Figure(layout=FigureLayout(height='420px'))